In [ ]:
import sys
import os
from pathlib import Path

# 定位项目根（project/）
PROJECT_ROOT = Path.cwd().parent 
# 把 src 加入 sys.path，这样可以直接 import 包
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

from AstroAgent.core.config.all_config import AllConfig
from AstroAgent.core.runtime.runtime_container import RuntimeContainer

from AstroAgent.manager.runtime.state_manager import SpectroStateFactory

In [ ]:
configs = AllConfig.from_env()
params_config = configs.params
io_config = configs.io
batch_config = configs.batch
runtime = RuntimeContainer(configs)

factory = SpectroStateFactory(configs)

In [ ]:
input_dir = io_config.input_dir
output_dir = io_config.output_dir

if io_config.run=='b':
    if batch_config.is_batch_mode():
        image_ids = batch_config.generate_ids()
        # logging.info(f\Batch mode enabled, total images: {len(image_ids)}\)
    else: 
        raise ValueError('Batch mode is enabled but start/end are not set')
else:
    image_ids = [io_config.image_name]
    # logging.info(\Single image mode enabled\)

In [ ]:
img_name = image_ids[0]
# image_path = os.path.join(input_dir, f\{img_name}.png\)

state = factory.create(
        image_name=img_name,
        input_dir=input_dir,
        output_dir=output_dir
)

In [ ]:
from AstroAgent.agents.multi_agents.VisualInterpreter import VisualInterpreter

a = VisualInterpreter(runtime)
await a.run(state)
print('done')

In [ ]:
from AstroAgent.agents.multi_agents.RuleAnalyst import RuleAnalyst

b = RuleAnalyst(runtime)
await b.run(state)
print('done')

In [ ]:
from AstroAgent.agents.multi_agents.AnalysisAuditor import AnalysisAuditor
from AstroAgent.agents.multi_agents.RefinementAssistant import RefinementAssistant

c = AnalysisAuditor(runtime)
d = RefinementAssistant(runtime)

for i in range(3):
    await c.run(state)
    await d.run(state)

In [ ]:
from AstroAgent.agents.multi_agents.SynthesisHost import SynthesisHost

e = SynthesisHost(runtime)
await e.run(state)